In [26]:
from frgpascal.experimentaldesign.tasks import *
from frgpascal.experimentaldesign.protocolwriter import generate_ot2_protocol
from frgpascal.hardware.sampletray import SampleTray, AVAILABLE_VERSIONS as sampletray_versions
from frgpascal.hardware.liquidlabware import TipRack, LiquidLabware, AVAILABLE_VERSIONS as liquid_labware_versions
from frgpascal.hardware.hotplate import AVAILABLE_VERSIONS as hotplate_versions
import frgpascal.experimentaldesign.characterizationtasks as ct
# from frgpascal.analysis import photoluminescence as PL 
from frgpascal.bridge import PASCALAxQueue

-----

# Define hardware for this experiment

## Sample Storage Trays
These are the trays used to load samples in/out of the glovebox

In [7]:
print('Available Sample Tray Versions:')
for v in sampletray_versions:
    print(f'\t{v}')

Available Sample Tray Versions:
	storage_v3
	storage_v2
	storage_v1


In [32]:
sample_tray = SampleTray(
    name='Tray1',
    version='storage_v1',
    gantry=None,
    gripper=None,
    p0=[0,0,0]
)

## Liquid Labware 

Include _all possible_ liquid storage + pipette tipracks. Later on we will narrow this down to what is required once we know what solutions and volumes we need.

Versions are defined by the same json files used to define custom labware for the Opentrons2 liquid handler. 
New labware can be defined by following https://support.opentrons.com/en/articles/3136504-creating-custom-labware-definitions

The location of the labware/tiprack on the opentrons deck must be specified as well.


| Deck | Slot | Layout |
| ----------- | ----------- | ----------- |
| 10      | 11       | _Trash_ |
| 7   | 8        | 9 |
|4 | 5 | 6|
|1 | _Spin_ | _Coater_|


For partially consumed tip racks, the `starting_tip` argument can be used to specify the first available tip. Tips before this assumed to be used up. (moves down->right, like A1, B1...H1, A2, B2 etc). If this argument is omitted, we assume to start at tip A1

In [9]:
print('\nAvailable Liquid Labwares:')
for v in liquid_labware_versions:
    print(f'\t{v}')


Available Liquid Labwares:
	frg_12_wellplate_15000ul
	sartorius_safetyspace_tiprack_200ul
	greiner_96_wellplate_360ul
	frg_spincoater_v1
	frg_24_wellplate_4000ul


In [10]:
tipracks = [
    TipRack(
        version='sartorius_safetyspace_tiprack_200ul', 
        deck_slot=9,
        starting_tip="F8"
    ),
    TipRack(
        version='sartorius_safetyspace_tiprack_200ul', 
        deck_slot=8,
        starting_tip="A2"
    ),
    TipRack(
        version='sartorius_safetyspace_tiprack_200ul', 
        deck_slot=11,
        starting_tip="A2"
    ),
    ]

the `starting_well` argument can be used to similarly specify the first available well on the tray. This is useful for mixing on partially used well plates. Note that the use order of wells differs from tips (moves right->down, like A1, A2,...A12, B1, B2 etc)

In [12]:
plate96 = LiquidLabware(
    name='96_Plate1',
    version='greiner_96_wellplate_360ul',
    deck_slot=5,
    starting_well="B2" 
)

tray4ml = LiquidLabware(
    name='4mL_Tray1',
    version='frg_24_wellplate_4000ul',
    deck_slot=6
)

solution_storage = [
    plate96,
    tray4ml   
]

#sort by volume,name
solution_storage.sort(key=lambda labware: labware.name)
solution_storage.sort(key=lambda labware: labware.volume)
print('Priority Fill Order:')
for ss in solution_storage: print(ss)

Priority Fill Order:
<LiquidLabware> 96_Plate1, 0.36 mL volume, 96 wells
<LiquidLabware> 4mL_Tray1, 4.0 mL volume, 24 wells


-----

# Define Available Solutions

Solutions are defined using `Solution` class instances.

Chemical formula is defined with underscores between each component. If no coefficient is provided, assumes =1. Example:

``` 
SolutionRecipe(
        solutes='MA_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    )
```
Note that you can use parentheses to simplify the formulae.
``` 
SolutionRecipe(
        solutes='MA_Pb_(I0.8_Br0.2)3',
        solvent='DMF9_DMSO1',
        molarity=1
    )
```

Antisolvents are also defined using `Solution` class instances. Solutes are left empty, and molarity is unused so can be left as 1/whatever number you like.

Example:
``` 
Solution(
        solutes='',
        solvent='Chlorobenzene',
        molarity=1
    )
```

Solutions that will be interchanged (for example, different absorber solutions for compositional search) should be stored in a list. This list will be used later to permute sample process variables


HTL

In [18]:
tray4ml.unload_all()

In [19]:
htl_ptaa = Solution(
    solutes='PTAA',
    solvent='Toluene',
    molarity=1.5, #mg/mL
    )
tray4ml.load(htl_ptaa, well='A2')

pfn_br = Solution(
    solutes = 'PFNBr',
    solvent='Methanol',
    molarity=0.5, #mg/mL
)
tray4ml.load(pfn_br, 'A3')

'A3'

Absorber 

In [20]:
absorber = Solution(
        solutes='FA0.78_Cs0.12_MA0.1_(Pb_(I0.8_Br0.10_Cl0.10)3)1.09',
        solvent='DMF3_DMSO1',
        molarity=1.2,
)
tray4ml.load(absorber, well='A1')

antisolvent_ma = Solution(
    solvent='MethylAcetate',
)
tray4ml.load(antisolvent_ma, well='D1')

'D1'

ETL

In [21]:
etl_pcbm = Solution(
    solutes='PCBM',
    solvent='Chlorobenzene',
    molarity=30, #mg/mL
)
tray4ml.load(etl_pcbm, well='B1')

'B1'

## Plan all possible characterization tasks

Create `Characterize` tasks

In [27]:
char_noPL = Characterize(
    tasks = [
        ct.TransmissionSpectroscopy(),
        ct.Brightfield(),
        ct.Darkfield(),
    ],
)

char_PL = Characterize(
    tasks = [
        ct.TransmissionSpectroscopy(),
        ct.PLSpectroscopy(),
        ct.Brightfield(),
        ct.Darkfield(),
    ],
)

ALL_CHAR_TASKS = [char_noPL, char_PL]

Export baseline requirements for maestro

In [28]:
baselines_required = {}

for taskobj in ALL_CHAR_TASKS:
    task = taskobj.to_dict()
    for ctask in task["details"]["characterization_tasks"]:
        if ctask["name"] not in baselines_required:
            baselines_required[ctask["name"]] = set()

        if "exposure_time" in ctask["details"]:
            baselines_required[ctask["name"]].add(
                ctask["details"]["exposure_time"]
                )
        if "exposure_times" in ctask["details"]:            
            for et in ctask["details"]["exposure_times"]:
                baselines_required[ctask["name"]].add(et)

baselines_required = {k:list(v) for k,v in baselines_required.items()}

out = {
    'baselines_required': baselines_required
} 
fname = f'maestronetlist_externalcontrol.json'
with open(fname, 'w') as f:
    json.dump(out, f, indent=4, sort_keys=True)
print(f'Characterization Baseline Instructions dumped to "{fname}"')

Characterization Baseline Instructions dumped to "maestronetlist_externalcontrol.json"


### Export protocol for OT2 liquid handler

Now that we have defined all the labware + solutions that go onto the liquid handler, we need to passs this info to the liquid handler itself. This protocol should be executed prior to starting maestro + the experiment.

In [23]:
generate_ot2_protocol(
    title='OT2Protocol_ExternalControl',
    mixing_netlist= {},
    labware=solution_storage,
    tipracks=tipracks
    )

OT2 protocol dumped to "./OT2PASCALProtocol_OT2Protocol_ExternalControl.py"


-----

# Functions to generate our samples

In [34]:
def build_PTAA_PFNBr(rpm_ptaa, volume_ptaa, droptime_ptaa, rpm_pfnbr, volume_pfnbr, droptime_pfnbr) -> Sample:
    spincoat_htl = Spincoat( 
        steps = [
            [rpm_ptaa,500,30], #speed (rpm), acceleration (rpm/s), duration (s)
        ],
        drops = [
            Drop(
                solution=htl_ptaa,
                volume=volume_ptaa,
                time=droptime_ptaa,
                reuse_tip = True,
                pre_mix = (3,100)
            )
        ]
    )

    spincoat_pfnbr = Spincoat(
        steps = [
            [rpm_pfnbr,2000,40], #speed (rpm), acceleration (rpm/s), duration (s)
        ],
        drops = [
            Drop(
                solution=etl_pcbm,
                volume=volume_pfnbr,
                time=droptime_pfnbr,
                pre_mix = (3,100)
            )
        ]
    )

    sample = Sample(
        name = 'will_be_overwritten_by_bridge',
        substrate='FTO',
        worklist = [
            spincoat_htl,
            char_noPL,
            spincoat_pfnbr,
            char_noPL,
        ],
        storage_slot = {
            "tray": sample_tray.name, 
            "slot": sample_tray.load('will_be_overwritten_by_bridge')
            },
    )

    return sample

-----

Make sure Maestro is operating in external control mode before running past this point

In [ ]:
bridge = PASCALAxQueue()


Add Samples

In [ ]:
for i in range(3):
    sample = build_PTAA_PFNBr(
        rpm_ptaa=5000,
        volume_ptaa=30,
        droptime_ptaa=-5,
        rpm_pfnbr=5000,
        volume_pfnbr=30,
        droptime_pfnbr=5 
    )
    bridge._send_sample_to_maestro(sample)